In [2]:
import numpy as np

### Basic Setting
Say the sample space of X is
\begin{equation}
    \{H,T\}
\end{equation}
and transition probability 
\begin{align*}
    P(T|H) &= 0.2~~~P(H|H)=0.8\\
    P(H|T) &= 0.7~~~P(T|T)=0.3
\end{align*} 
and the sample space of Y is
\begin{equation}
    \{R,G,B\}
\end{equation}
and the emission probability is
\begin{align*}
    P(R|H) &= 0.25~~~P(G|H)=0.25~~~P(B|H)=0.5\\
    P(R|T) &= 0.4~~~P(G|T)=0.4~~~P(B|T)=0.2
\end{align*}  

### Graphical Model
![alt text](./images/coin_ball_example.svg "Title")

### Part 1: Brute-Force Solution

#### $P(\textbf{Y})=\sum_{\textbf{X}}P(\textbf{Y}|\textbf{X})P(\textbf{X})$

In [3]:
latent_sequences = ['HHH','HHT','HTH','THH','THT','TTH','HTT','TTT']
observed_sequences = ['R','G']
pX = [0.32, 0.08, 0.07, 0.28, 0.07, 0.105, 0.03, 0.045]
pY_given_X = [0.0625, 0.1, 0.1, 0.0625, 0.1, 0.1, 0.16, 0.16]
pY = np.dot(pX, pY_given_X)
pY

0.082

### Part 2: Transition Matrix $A$, Emission Matrix $\hat{\textbf{y}}$
\begin{equation}
A = 
\begin{bmatrix}
P(H|H) & P(T|H) \\
P(H|T) & P(T|T)
\end{bmatrix}
=
\begin{bmatrix}
0.8 & 0.2 \\
0.7 & 0.3
\end{bmatrix}
\end{equation}

In [4]:
A = np.array([[0.8, 0.2],
              [0.7, 0.3]])

\begin{equation}
\hat{\textbf{y}}_{t} = 
\begin{bmatrix}
P(y_t|H) & 0 \\
0 & P(y_t|T)
\end{bmatrix}
\end{equation}
 the emission probability is
\begin{align*}
    P(R|H) &= 0.25~~~P(G|H)=0.25~~~P(B|H)=0.5\\
    P(R|T) &= 0.4~~~P(G|T)=0.4~~~P(B|T)=0.2
\end{align*}  

In [5]:
y_dict = {
    'R': np.array([[0.25, 0],
                   [0, 0.4]]),
    'G': np.array([[0.25, 0],
                   [0, 0.4]]),
    'B': np.array([[0.5, 0],
                   [0, 0.2]])
}

### Part 3: Alpha-Recursion

#### $\left<\alpha_{t_0}|x_{t_0}\right>=p(x_{t_0})$

In [19]:
alpha_t0 = np.array([[0.5, 0.5]])

#### $\left<\alpha_{t_0}|A|x_{t_1}\right>$

In [20]:
alpha_t0_A = np.dot(p0, A)
alpha_t0_A

array([[0.75, 0.25]])

#### $\left<\alpha_{t_1}|x_{t_1} \right> = \left<\alpha_{t_0}|A\hat{\textbf{y}}_1|x_{t_1}\right>$

In [22]:
photon_operator = y_dict['R']
alpha_t1 = np.dot(p0_A,photon_operator)
alpha_t1

array([[0.1875, 0.1   ]])

#### $\left<\alpha_{t_1}|A|x_{t_2}\right>$

In [23]:
alpha_t1_A = np.dot(alpha_t1, A)
alpha_t1_A

array([[0.22  , 0.0675]])

#### $\left<\alpha_{t_2}|x_{t_2} \right> = \left<\alpha_{t_1}|A\hat{\textbf{y}}_2|x_{t_2}\right>$

In [24]:
photon_operator = y_dict['G']
alpha_t2 = np.dot(alpha_t1_A, photon_operator)
alpha_t2

array([[0.055, 0.027]])

#### $P(\textbf{Y}) = \left<\alpha_{t_2}|\beta_{t_{\text{exp}}}\right> = \left<\alpha_{t_0}|A\hat{\textbf{y}}_1 A\hat{\textbf{y}}_2|\beta_{t_{\text{exp}}}\right>$
where
\begin{equation}
\left|\beta_{t_{\text{exp}}}\right> = \begin{bmatrix} 1 \\ 1\end{bmatrix}
\end{equation}

In [25]:
beta_t_exp = np.array([[1],[1]])
pY_by_alpha = np.dot(alpha_t2, beta_t_exp)
pY_by_alpha

array([[0.082]])

### Part 4: Beta-Recursion
#### $\left| \beta_2 \right>$

In [6]:
beta_2 = np.array([[1],[1]])

#### $\hat{\textbf{y}}_2\left| \beta_2 \right>$

In [10]:
photon_operator = y_dict['G']
y_beta_2 = np.dot(photon_operator, beta_2)
y_beta_2

array([[0.25],
       [0.4 ]])

#### $\left|\beta_1 \right>=A\hat{\textbf{y}}_2\left| \beta_2 \right>$
\begin{equation}
\begin{bmatrix}
P(y_2=G|x_1=H) \\ P(y_2=G|x_1=T)
\end{bmatrix}
\end{equation}

\begin{equation}
P(G_2|H_1) = P(H_2|H_1)P(G_2|H_2) + P(T_2|H_1)P(G_2|T_2)
\end{equation}
\begin{equation}
P(G_2|T_1) = P(H_2|T_1)P(G_2|H_2) + P(T_2|T_1)P(G_2|T_2)
\end{equation}

In [11]:
beta_1 = np.dot(A, y_beta_2)
beta_1

array([[0.28 ],
       [0.295]])

In [9]:
p_g2_given_h1 = (0.8 * 0.25) + (0.2 * 0.4)
print(p_g2_given_h1)
p_g2_given_t1 = (0.7 * 0.25) + (0.3 * 0.4)
print(p_g2_given_t1)

0.28
0.295
